In [1]:
import os
import re
import numpy as np
import pandas as pd
import math
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import operator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer

### Unknown Words Addition

In [2]:
# Used to add UNK to the training corpus
def unknown_words(corpus):
    for i in range(len(corpus)):
        toss = np.random.binomial(size=1, n=1, p= 0.01)
        if toss == 1:
            corpus[i] = 'UNK'
    
    return corpus

In [3]:
# Used to add UNK in validation and Testing corpus
def add_unknown_words(corpus, unigram_count):
    for i in range(len(corpus)):
        if corpus[i] not in unigram_count:
            corpus[i] = 'UNK'
    return corpus

### Importing Data from Training Set and Fitting NB model

In [10]:
filename = 'truthful.txt'
f = open(os.getcwd() + "/DATASET/train/" + filename, "r")

stop_words = ['the','and','a','to','was','i','for','it','is','be','as','so','are','all','when']
                    # 'when' was the last word which gave accuracy. 'not', 'in','at'
text = []
ytruth = []

# Implementation of Linguistic Features:
lemmer=WordNetLemmatizer()

for line in f:
    line = line.lower()
#     line = line.replace(" '", "")
#     word_list  = line.split()
#     word_list = unknown_words(word_list)
#     line = ' '.join(word[0] for word in word_list)
    line = '<s>' + ' ' + line
    line=' '.join([lemmer.lemmatize(word) for word in line.split()])
    text.append(line)
    ytruth.append(0)
f.close()
# print(text[0])

filename = 'deceptive.txt'
f = open(os.getcwd() + "/DATASET/train/" + filename, "r")

for line in f:
    line = line.lower()
#     line = line.replace(" '", "")
#     word_list  = line.split()
#     word_list = unknown_words(word_list)
#     line = ' '.join([word for word in word_list])
    line = '<s>' + ' ' + line
    line=' '.join([lemmer.lemmatize(word) for word in line.split()])
    text.append(line)
    ytruth.append(1)
f.close()

# train_corpus = ' '.join(line[0] for line in text)

# vectorizer = CountVectorizer(analyzer = 'word')
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words = stop_words)     # Bigrams, token_pattern=r'\b\w+\b'
# vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words = stop_words)     # Bigrams, token_pattern=r'\b\w+\b'
features_X_train = vectorizer.fit_transform(text)
word_types = vectorizer.get_feature_names()
ytruth_train = ytruth
clf = MultinomialNB(alpha = 1, class_prior=None, fit_prior=True)
# clf = BernoulliNB(alpha = 0.005, class_prior=None, fit_prior=None)
clf.fit(features_X_train, ytruth_train)
features_X_train

<1024x69227 sparse matrix of type '<type 'numpy.int64'>'
	with 199342 stored elements in Compressed Sparse Row format>

### Importing Data from Validation Set and Predicting using NB model

In [12]:
# We ran this code for different parameters on validation dataset to get different accuracies, in report, Table 2: 

filename = 'truthful.txt'
f = open(os.getcwd() + "/DATASET/validation/" + filename, "r")

text = []
ytruth = []

for line in f:
    line = line.lower()
#     line = line.replace(" '", "")
#     word_list  = line.split()
#     word_list = add_unknown_words(word_list, train_corpus)
#     line = ' '.join(word[0] for word in word_list)
    line = '<s>' + ' ' + line
    line=' '.join([lemmer.lemmatize(word) for word in line.split()])
    text.append(line)
    ytruth.append(0)
f.close()

filename = 'deceptive.txt'
f = open(os.getcwd() + "/DATASET/validation/" + filename, "r")

for line in f:
    line = line.lower()
#     line = line.replace(" '", "")
#     word_list  = line.split()
#     word_list = add_unknown_words(word_list, train_corpus)
#     line = ' '.join(word[0] for word in word_list)
    line = '<s>' + ' ' + line
    line=' '.join([lemmer.lemmatize(word) for word in line.split()])
    text.append(line)
    ytruth.append(1)
f.close()

features_X_valid = vectorizer.transform(text)
word_types = vectorizer.get_feature_names()
ytruth_valid = ytruth
features_X_valid
ypred = clf.predict(features_X_valid)

acc_score = accuracy_score(ytruth_valid, ypred)
print('Accuracy Score: %s'%(acc_score))

Accuracy Score: 0.921875


### Importing data from Test set and getting predicted classes

In [37]:
filename = 'test.txt'
f = open(os.getcwd() + "/DATASET/test/" + filename, "r")

text = []

for line in f:
    line = line.lower()
    line = '<s>' + ' ' + line
    line=' '.join([lemmer.lemmatize(word) for word in line.split()])
    text.append(line)
    
f.close()
features_X_test = vectorizer.transform(text)
ypred_test = clf.predict(features_X_test)

output = pd.DataFrame(ypred_test)
output.columns = ['Prediction']
output.index.name = 'Id'
path = os.getcwd() + '/result_NB.csv'
output.to_csv(path)
output.head()

,Prediction
Id,
0,1
1,0
2,0
3,1
4,0


In [63]:
import nltk
nltk.download('wordnet')
  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abhishek\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True